In [1]:
import json
import requests

In [6]:
# Porijõgi station code
station_code = 'SJA3124000'

In [7]:
# Read JSON file and get a list with a single dictionary
with open('data/estmodel20211101100645.json', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
# POST request using the JSON data
r = requests.post('https://estmodel.envir.ee/', json=data[0])

In [4]:
r.status_code

200

In [9]:
# Test writing a JSON string to file
with open('data/{}_response.json'.format(station_code), 'w', encoding='utf-8') as file:
    json_string = json.dumps(r.json(), ensure_ascii=False, indent=4)
    print(json_string, file=file)
    file.close()

In [10]:
# def get_site_code(catchment):
#     site_code = catchment.get('code')
#     return site_code

In [11]:
# def get_subcatchments(catchment):
#     subcatchments = catchment['subcatchments']
#     return subcatchments

Tried creating a function for recursively looking through the JSON object and getting to where the Porijõgi station data is located at. The info is hidden somewhere inside a nested structure of dictionaries and lists, so to make the procedure more robust and usable regardless of the level of the station there needs to be ways to switch the search target between a dictionary and list.

In [47]:
def extract_json(json_object, station_code):
    if isinstance(json_object, dict) and json_object:
        for k, v in json_object.items():
            if json_object.get('code') == station_code:
                print('Station name: {}'.format(json_object['name']))
                print('Station code: {}'.format(station_code))
                factor_dict = json_object['adjustments'][0]
                print('Adjustment factor for parameter {}: {}'.format(factor_dict.get('parameter').upper(), factor_dict.get('factor')))
                for i in range(len(json_object['diffuseSources'])):
                    if json_object['diffuseSources'][i]['type'] in ['arableland', 'forest', 'pasture', 'water']:
                        print(json_object['diffuseSources'][i]['type'])
                        print(json_object['diffuseSources'][i]['estimates'])
                print('\n')
            else:
                extract_json(v, station_code)
    elif isinstance(json_object, list) and json_object:
        for item in json_object:
            extract_json(item, station_code)

Currently the function prints out the estimates of the model per LULC type 14 times, so obviously something is wrong with the loop. However, extracting the estimates for atmospheric deposition, etc from these should not be too difficult. The estimates could be written into a CSV file or something like that.

The more difficult part is writing the new adjustment factor into a JSON file in the correct position, save that as a new JSON or simply a dictionary in memory and submit it as a POST request.

In [48]:
with open('data/estmodel20211101120617.json', encoding='utf-8') as file:
    data = json.load(file)
    extract_json(data, station_code)

Station name: Porijõgi: Reola – Vana-Kuuste tee (Uhti)
Station code: SJA3124000
Adjustment factor for parameter TN: 1.1
arableland
[{'parameter': 'tn', 'anthropogenicDischarge': 42731.630588040345, 'atmosphericDeposition': 14.332421164742346, 'naturalDischarge': 15622.057741145984, 'retention': 31584.456203186222}, {'parameter': 'tp', 'anthropogenicDischarge': 556.7871198095736, 'atmosphericDeposition': 0.26900207582870955, 'naturalDischarge': 566.8748051564348, 'retention': 574.9934756244065}]
pasture
[{'parameter': 'tn', 'anthropogenicDischarge': 0, 'atmosphericDeposition': 1.102500853767894, 'naturalDischarge': 1201.7042898232755, 'retention': 650.8700811275247}, {'parameter': 'tp', 'anthropogenicDischarge': 0, 'atmosphericDeposition': 0.02069259721421392, 'naturalDischarge': 43.606027863730674, 'retention': 22.31905807054934}]
forest
[{'parameter': 'tn', 'anthropogenicDischarge': 0, 'atmosphericDeposition': 13.311040995705788, 'naturalDischarge': 14508.773405377067, 'retention': 78